# [LangSmith] Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

We'll also provide our OpenAI key, as well as our Cohere API key.

In [2]:
import gc;

In [3]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env")

True

In [4]:
def check_if_env_var_is_set(env_var_name: str, human_readable_string: str = "API Key"):
    api_key = os.getenv(env_var_name)
  
    if api_key:
       print(f"{env_var_name} is present")
    else:
      print(f"{env_var_name} is NOT present, paste key at the prompt:")
      os.environ[env_var_name] = getpass.getpass(f"Please enter your {human_readable_string}: ")

In [5]:
import os
import getpass

# os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

check_if_env_var_is_set("OPENAI_API_KEY", "OpenAI API key")

OPENAI_API_KEY is present


In [6]:
# os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

check_if_env_var_is_set("COHERE_API_KEY", "Cohere API key")

COHERE_API_KEY is present


## Task 2: Data Collection and Preparation

We'll be using our Loan Data once again - this time the strutured data available through the CSV!

### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

In [7]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

loader = CSVLoader(
    file_path=f"./data/complaints.csv",
    metadata_columns=[
      "Date received", 
      "Product", 
      "Sub-product", 
      "Issue", 
      "Sub-issue", 
      "Consumer complaint narrative", 
      "Company public response", 
      "Company", 
      "State", 
      "ZIP code", 
      "Tags", 
      "Consumer consent provided?", 
      "Submitted via", 
      "Date sent to company", 
      "Company response to consumer", 
      "Timely response?", 
      "Consumer disputed?", 
      "Complaint ID"
    ]
)

loan_complaint_data = loader.load()

for doc in loan_complaint_data:
    doc.page_content = doc.metadata["Consumer complaint narrative"]

Let's look at an example document to see if everything worked as expected!

In [8]:
loan_complaint_data[0]

Document(metadata={'source': './data/complaints.csv', 'row': 0, 'Date received': '03/27/25', 'Product': 'Student loan', 'Sub-product': 'Federal student loan servicing', 'Issue': 'Dealing with your lender or servicer', 'Sub-issue': 'Trouble with how payments are being handled', 'Consumer complaint narrative': "The federal student loan COVID-19 forbearance program ended in XX/XX/XXXX. However, payments were not re-amortized on my federal student loans currently serviced by Nelnet until very recently. The new payment amount that is effective starting with the XX/XX/XXXX payment will nearly double my payment from {$180.00} per month to {$360.00} per month. I'm fortunate that my current financial position allows me to be able to handle the increased payment amount, but I am sure there are likely many borrowers who are not in the same position. The re-amortization should have occurred once the forbearance ended to reduce the impact to borrowers.", 'Company public response': 'None', 'Company'

In [9]:
gc.collect()

38

## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "LoanComplaints".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [10]:
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient, models
from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings

small_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    loan_complaint_data,
    small_embeddings,
    location=":memory:",
    collection_name="LoanComplaints"
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [11]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [12]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-4.1-nano` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [13]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4.1-nano",                       
    temperature=0.1,      # Lower temperature for more consistent outputs
    request_timeout=120   # Longer timeout for complex operations
)

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [14]:
%%time
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

CPU times: user 19.9 ms, sys: 3.28 ms, total: 23.2 ms
Wall time: 33.2 ms


Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [15]:
%%time
# naive_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

CPU times: user 5 μs, sys: 1e+03 ns, total: 6 μs
Wall time: 10 μs


In [16]:
%%time
# naive_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

CPU times: user 4 μs, sys: 1 μs, total: 5 μs
Wall time: 9.06 μs


In [17]:
%%time
# naive_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

CPU times: user 8 μs, sys: 1e+03 ns, total: 9 μs
Wall time: 14.5 μs


Overall, this is not bad! Let's see if we can make it better!

In [18]:
gc.collect()

48

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [19]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(loan_complaint_data, )

We'll construct the same chain - only changing the retriever.

In [20]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [21]:
%%time
# bm25_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

CPU times: user 5 μs, sys: 1e+03 ns, total: 6 μs
Wall time: 9.3 μs


In [22]:
%%time
# bm25_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

CPU times: user 8 μs, sys: 1e+03 ns, total: 9 μs
Wall time: 24.6 μs


In [23]:
%%time
# bm25_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

CPU times: user 4 μs, sys: 0 ns, total: 4 μs
Wall time: 8.58 μs


It's not clear that this is better or worse, if only we had a way to test this (SPOILERS: We do, the second half of the notebook will cover this)

In [24]:
gc.collect()

0

#### ❓ Question #1:

Give an example query where BM25 is better than embeddings and justify your answer.

#### ✅ Answer:

BM25, a traditional full-text search ranking function, is particularly effective when dealing with queries that rely heavily on exact term matching, term frequency, and inverse document frequency (TF-IDF) principles.

BM25 is generally better suited for scenarios where exact keyword matching is essential, such as in e-commerce search engines, document retrieval systems, and legal e-discovery.

Additionally, BM25 is often used in hybrid search systems alongside vector search to create a more comprehensive understanding of both semantic meaning and keyword importance.

Here are a couple of queries where the exact matching terms in the document would be essential to prevent a lot of results with noise and near close terms but not close enough:

- "Find documents about COVID-19 vaccine side effects in patients with diabetes"
  - the key terms here COVID-19 vaccine and diabetes are were the focus is in the query
- "Best practices for data backup in 2025"
  - It includes specific terms like "data backup" and "2025" that are likely to appear verbatim in relevant documents.
  - BM25 can effectively leverage term frequency (e.g., how often "data backup" appears in a document) and document length normalization to rank documents accurately. The query does not heavily rely on semantic similarity but rather on the presence and frequency of exact keywords.
  - In contrast, dense embeddings might struggle if the training data does not include similar phrasing or if the semantic model does not strongly associate "best practices" with "data backup" in the context of 2025.

Embeddings, on the other hand, are better suited for capturing semantic relationships between words and documents. If embeddings were used in the above scenarios or use-cases, the precision of the results would not be as accurate as with BM25.


### Addendum

_**Sparse Embeddings** are high-dimensional vectors where most values are zero, with only a few non-zero values representing specific features or tokens that are present, making them memory-efficient and interpretable but limited to explicit feature representation._

_**Dense Embeddings** are vectors where most or all dimensions have non-zero values, creating rich, continuous representations that capture complex semantic relationships and contextual meaning, but require more storage and are less interpretable._

_**Key Difference:** Sparse embeddings work like "on/off switches" for specific features (like one-hot encoding or TF-IDF), while dense embeddings work like "semantic fingerprints" where every dimension contributes to the overall meaning representation - sparse focuses on explicit presence/absence, dense captures nuanced relationships._

___

_**Sparse Retrieval** uses exact keyword matching with algorithms like BM25, where documents are represented as sparse vectors containing only the specific terms that appear in them, making it excellent for precise term-based searches but limited to lexical matches._

_**Dense Retrieval** uses semantic embeddings where documents and queries are converted into dense vector representations that capture meaning and context, allowing it to find semantically similar content even when different words are used, but potentially missing exact keyword matches._

_**Key Difference:** Sparse retrieval excels at "what you search is what you get" with exact terms, while dense retrieval excels at "what you mean is what you get" through semantic understanding - which is why hybrid approaches combining both often work best._


## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [25]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-v3.5")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [26]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [27]:
%%time
# contextual_compression_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

CPU times: user 3 μs, sys: 1 μs, total: 4 μs
Wall time: 6.44 μs


In [28]:
%%time
# contextual_compression_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

CPU times: user 4 μs, sys: 1e+03 ns, total: 5 μs
Wall time: 6.68 μs


In [29]:
%%time
# contextual_compression_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

CPU times: user 6 μs, sys: 1e+03 ns, total: 7 μs
Wall time: 11.9 μs


We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

In [30]:
gc.collect()

40

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [31]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

In [32]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [33]:
%%time
# multi_query_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

CPU times: user 5 μs, sys: 1e+03 ns, total: 6 μs
Wall time: 11.2 μs


In [34]:
%%time
# multi_query_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

CPU times: user 8 μs, sys: 0 ns, total: 8 μs
Wall time: 13.1 μs


In [35]:
%%time
# multi_query_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 5.72 μs


In [36]:
gc.collect()

0

#### ❓ Question #2:

Explain how generating multiple reformulations of a user query can improve recall.

#### ✅ Answer:

Multiple reformulations improve recall because relevant documents may use different terminology than the original query, and each reformulation can surface documents the others miss (different phrasings in multiple reformulations of a query can match different relevant documents).

In other words, multiple reformulations approach the same query from different angles/facets, leading to retrieval of documents covering those various angles. This increases the confluence of documents around the common theme while capturing variations in terminology and perspective, thereby enhancing retrieval scope.

And since such retrievers that use multiple reformulations would follow the below steps:

  1. Generates multiple query variations from the original query using an LLM
  2. Retrieves documents for each variation (each gets k results)
  3. Deduplicates and merges the results from all queries
  4. Returns the final deduplicated set

The return results from multiple reformulations would be more beneficial as a retrieval process.

An example would be "machine learning algorithms" vs "AI models" retrieves different relevant documents but around the same or similar theme.

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [37]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter

parent_docs = loan_complaint_data
child_splitter = RecursiveCharacterTextSplitter(chunk_size=750)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [38]:
vectorstore.client.create_collection(
  collection_name="full_documents",
  vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = Qdrant(
  client=vectorstore.client,     # ✅ Reuse existing client
  embeddings=small_embeddings,         # ✅ Reuse embeddings
  collection_name="full_documents"
)

/tmp/ipykernel_540/921369447.py:6: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  parent_document_vectorstore = Qdrant(


Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [39]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [93]:
parent_document_retriever.add_documents(parent_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [94]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [95]:
%%time
# parent_document_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

CPU times: user 7 μs, sys: 1 μs, total: 8 μs
Wall time: 13.4 μs


In [96]:
%%time
# parent_document_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

CPU times: user 6 μs, sys: 1 μs, total: 7 μs
Wall time: 10.3 μs


In [97]:
%%time
# parent_document_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

CPU times: user 4 μs, sys: 0 ns, total: 4 μs
Wall time: 6.2 μs


Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

In [98]:
gc.collect()

366

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [99]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [100]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [101]:
%%time
# ensemble_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

CPU times: user 11 μs, sys: 0 ns, total: 11 μs
Wall time: 17.6 μs


In [102]:
%%time
# ensemble_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

CPU times: user 10 μs, sys: 0 ns, total: 10 μs
Wall time: 15.7 μs


In [103]:
%%time
# ensemble_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

CPU times: user 4 μs, sys: 1 μs, total: 5 μs
Wall time: 6.68 μs


In [104]:
gc.collect()

0

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

The `breakpoint_threshold_type` parameter controls when the semantic chunker creates chunk boundaries based on embedding similarity between sentences:

**Four Threshold Types:**

1. _"percentile" (default)_
- Splits when sentence embedding distance exceeds the 95th percentile of all distances
- Effect: Creates chunks at the most semantically distinct boundaries
- Behavior: More conservative splitting, larger chunks

2. _"standard_deviation"_
- Splits when distance exceeds 3 standard deviations from mean
- Effect: Better predictable performance, especially for normally distributed content
- Behavior: More consistent chunk sizes

3. _"interquartile"_
- Uses IQR * 1.5 scaling factor to determine breakpoints
- Effect: Middle-ground approach, robust to outliers
- Behavior: Balanced chunk distribution

4. _"gradient"_
- Detects anomalies in embedding distance gradients
- Effect: Best for domain-specific/highly correlated content
- Behavior: Finds subtle semantic transitions

**Impact:** _The threshold type determines sensitivity to semantic changes - more sensitive types create smaller, more focused chunks while less sensitive types create larger, more comprehensive chunks._

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [105]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    small_embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [106]:
%%time
semantic_documents = semantic_chunker.split_documents(loan_complaint_data[:20])

CPU times: user 840 ms, sys: 95.3 ms, total: 935 ms
Wall time: 2min 21s


Let's create a new vector store.

In [107]:
vectorstore.client.create_collection(
  collection_name="Loan_Complaint_Data_Semantic_Chunks",
  vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

semantic_vectorstore = Qdrant(
  client=vectorstore.client,     # ✅ Reuse existing client
  embeddings=small_embeddings,         # ✅ Reuse embeddings
  collection_name="Loan_Complaint_Data_Semantic_Chunks"
)

# Add documents after creation
_ = semantic_vectorstore.add_documents(semantic_documents)

We'll use naive retrieval for this example.

In [108]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [109]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [110]:
# semantic_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

In [111]:
# semantic_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

In [112]:
# semantic_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

In [113]:
gc.collect()

520

#### ❓ Question #3:

If sentences are short and highly repetitive (e.g., FAQs), how might semantic chunking behave, and how would you adjust the algorithm?

#### ✅ Answer:

Short and highly repetitive sentences create _minimal embedding distance_ variations, making it difficult to detect _meaningful semantic_ boundaries.

Threshold Type Behaviors:

1. "percentile" (95th percentile)

- Behavior: Creates very few chunks since most distances are similar
- Issue: May group unrelated FAQ topics together
- Adjustment: Lower to 75-85th percentile to increase sensitivity

2. "standard_deviation" (3σ)

- Behavior: Performs poorly due to low variance in short, similar sentences
- Issue: Creates massive chunks with no meaningful breaks
- Adjustment: Reduce to 1-2 standard deviations for more splitting

3. "interquartile" (IQR × 1.5)

- Behavior: Most robust for FAQs due to outlier resistance
- Issue: Still may miss subtle topic transitions
- Adjustment: Reduce scaling factor to 0.8-1.0

4. "gradient" (anomaly detection)

- Behavior: Best performer - detects subtle topic shifts in repetitive content
- Issue: May be overly sensitive to minor variations
- Adjustment: Fine-tune threshold to 85-90th percentile

Conclusion: Use "gradient" with _85th percentile_ + minimum chunk size constraints + keyword-based post-processing to ensure FAQ topics remain grouped appropriately despite repetitive language patterns.

# 🤝 Breakout Room Part #2

#### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against eachother.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

In [114]:
### YOUR CODE HERE

In [115]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [116]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env")

True

In [63]:
def check_if_env_var_is_set(env_var_name: str, human_readable_string: str = "API Key"):
    api_key = os.getenv(env_var_name)
  
    if api_key:
       print(f"{env_var_name} is present")
    else:
      print(f"{env_var_name} is NOT present, paste key at the prompt:")
      os.environ[env_var_name] = getpass.getpass(f"Please enter your {human_readable_string}: ")

In [64]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
check_if_env_var_is_set("LANGCHAIN_API_KEY", "LangChain API key")
check_if_env_var_is_set("LANGSMITH_API_KEY", "LangSmith API key")
check_if_env_var_is_set("OPENAI_API_KEY", "OpenAI API key")

LANGCHAIN_API_KEY is present
LANGSMITH_API_KEY is present
OPENAI_API_KEY is present


In [65]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - ADwLC - {uuid4().hex[0:8]}"

In [49]:
# docs = loan_complaint_data.copy()
print(f"Original documents count: {len(loan_complaint_data)}")

filtered_docs = []
for doc in loan_complaint_data:
    narrative = doc.metadata.get("Consumer complaint narrative", "")
    if (len(narrative.strip()) < 100 or 
        narrative.count("XXXX") > 5 or 
        narrative.strip() in ["", "None", "N/A"]):
        continue

    doc.page_content = f"Customer Issue: {doc.metadata.get('Issue', 'Unknown')}\n"
    doc.page_content += f"Product: {doc.metadata.get('Product', 'Unknown')}\n"
    doc.page_content += f"Complaint Details: {narrative}"

    filtered_docs.append(doc)

print(f"Documents count after filtering: {len(filtered_docs)}")

Original documents count: 825
Documents count after filtering: 480


In [50]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
# generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_llm = LangchainLLMWrapper(ChatOpenAI(
    model="gpt-4.1-nano",  # Less capable than mini for reasoning tasks, but okay for the task
    temperature=0.1,      # Lower temperature for more consistent outputs
    request_timeout=120   # Longer timeout for complex operations
))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [53]:
gc.collect()

9057

In [54]:
# %%time
from ragas.testset.transforms import default_transforms, apply_transforms
transformer_llm = generator_llm
embedding_model = generator_embeddings

In [56]:
gc.collect()

0

In [57]:
import psutil

# Check memory usage
process = psutil.Process(os.getpid())
memory_mb = process.memory_info().rss / 1024 / 1024
print(f"Memory usage before generation: {memory_mb:.1f} MB")

Memory usage before generation: 477.9 MB


In [58]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(
                llm=generator_llm, embedding_model=embedding_model, 
                #knowledge_graph=loan_data_kg
)

In [60]:
%%time
testset = None
if not os.path.exists('golden-master.csv'):
    # testset = generator.generate(testset_size=10, query_distribution=query_distribution)

    testset = generator.generate_with_langchain_docs(
        loan_complaint_data[:20], testset_size=10,
    )
    testset.to_pandas()

CPU times: user 0 ns, sys: 73 μs, total: 73 μs
Wall time: 78.9 μs


In [61]:
# Check memory usage
process = psutil.Process(os.getpid())
memory_mb = process.memory_info().rss / 1024 / 1024
print(f"Memory usage after generation: {memory_mb:.1f} MB")

Memory usage after generation: 477.9 MB


In [62]:
gc.collect()

0

In [63]:
process = psutil.Process(os.getpid())
memory_mb = process.memory_info().rss / 1024 / 1024
print(f"Memory usage after gc.collect(): {memory_mb:.1f} MB")

Memory usage after gc.collect(): 477.9 MB


In [64]:
import pandas as pd

In [65]:
if testset:
    testset_df = testset.to_pandas()
    testset_df.to_csv('golden-master.csv', index=False)
else:
    testset_df = pd.read_csv('golden-master.csv')
testset_df    

,user_input,reference_contexts,reference,synthesizer_name
0,"As a consumer rights advocate, how should I ad...","[""Customer Issue: Dealing with your lender or ...",The complaint details indicate that the federa...,single_hop_specifc_query_synthesizer
1,What is the issue with Aidvantage in the borro...,['Customer Issue: Dealing with your lender or ...,The borrower submitted their IDR recertificati...,single_hop_specifc_query_synthesizer
2,How does FERPA protect student loan borrowers'...,['Customer Issue: Dealing with your lender or ...,FERPA protects personal and financial data by ...,single_hop_specifc_query_synthesizer
3,What concerns does the borrower have regarding...,['Customer Issue: Dealing with your lender or ...,The borrower is concerned because they were su...,single_hop_specifc_query_synthesizer
4,How can borrowers address issues related to th...,['Customer Issue: Dealing with your lender or ...,The customer reports difficulties with their s...,single_hop_specifc_query_synthesizer
5,How does the illegal student loan reporting af...,['<1-hop>\n\nCustomer Issue: Improper use of y...,The illegal student loan reporting and collect...,multi_hop_specific_query_synthesizer
6,How does the failure of XXXX and XXXX to condu...,['<1-hop>\n\nThis account was transferred to N...,"The misconduct by XXXX and XXXX, including the...",multi_hop_specific_query_synthesizer
7,How does the CFPB's involvement relate to the ...,['<1-hop>\n\nThis is a formal legal demand for...,The CFPB's involvement is highlighted through ...,multi_hop_specific_query_synthesizer
8,How does the violation of the FCRA relate to t...,['<1-hop>\n\nThis account was transferred to N...,The violation of the FCRA is related to the im...,multi_hop_specific_query_synthesizer
9,How does NelNet's repeated failure to process ...,"[""<1-hop>\n\nXX/XX/XXXX I increased the amount...",The context shows that despite multiple confir...,multi_hop_specific_query_synthesizer


In [66]:
from langsmith import Client

langsmith_client = Client(
    timeout_ms=60000,  # 60 seconds
    retry_config={"max_retries": 5}
)

dataset_name = "Loan Synthetic Data (s09)"

existing_datasets = langsmith_client.list_datasets()
dataset_exists = any(dataset.name == dataset_name for dataset in existing_datasets)

if dataset_exists:
  langsmith_dataset = langsmith_client.read_dataset(dataset_name=dataset_name)
  print(f"Using existing dataset: {dataset_name}")
else:
  langsmith_dataset = langsmith_client.create_dataset(
      dataset_name=dataset_name,
      description="Loan Synthetic Data (for s09 exercise)"
  )
  print(f"Created new dataset: {dataset_name}")

Using existing dataset: Loan Synthetic Data (s09)


In [67]:
gc.collect()

0

In [68]:
%%time
try:
  example_count = 0
  for example in langsmith_client.list_examples(dataset_id=langsmith_dataset.id):
      langsmith_client.delete_example(example.id)
      example_count += 1
  print(f"Deleted {example_count} examples")
except Exception as e:
  print(f"Error with list_examples: {e}")

for data_row in testset_df.iterrows():
  langsmith_client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )
print(f'Created {testset_df.shape[0]} in the "{langsmith_dataset.name}" dataset')

Deleted 10 examples
Created 10 in the "Loan Synthetic Data (s09)" dataset
CPU times: user 150 ms, sys: 16.5 ms, total: 167 ms
Wall time: 12.3 s


In [70]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(filtered_docs)

In [71]:
from langchain_openai import OpenAIEmbeddings

In [72]:
from langchain_community.vectorstores import Qdrant

small_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
try:
    vectorstore.client.create_collection(
      collection_name="Loan RAG (semantic)",
      vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
    )
except:
    pass

semantic_vectorstore = Qdrant(
    client=vectorstore.client,     # ✅ Reuse existing client
    embeddings=small_embeddings,         # ✅ Reuse embeddings
    collection_name="Loan RAG (semantic)"
)

_ = semantic_vectorstore.add_documents(rag_documents)

In [73]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [74]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [75]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini",
            temperature=0.1,      # Lower temperature for more consistent outputs
            request_timeout=120   # Longer timeout for complex operations
)

In [76]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

## LangSmith Evaluation Set-up

In [77]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

In [78]:
eval_llm = ChatOpenAI(model="gpt-4.1",
    temperature=0.1,      # Lower temperature for more consistent outputs
    request_timeout=120   # Longer timeout for complex operations                     
)

In [79]:
def prepare_context_ref(run, example):
  return {
      "prediction" : run.outputs["response"],
      "reference": example.outputs["answer"],
      "input" : example.inputs["question"]
  }
cot_qa_evaluator = LangChainStringEvaluator(
	"cot_qa",  config={"llm":eval_llm}, prepare_data=prepare_context_ref
)

def prepare_data_noref(run, example):
  return {
      "prediction" : run.outputs["response"],
      "input" : example.inputs["question"]
  }
unlabeled_dopeness_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria" : {
            "dopeness" : "Is the answer to the question dope, meaning cool - awesome - and legit?"
        },
        "llm" : eval_llm,
    },
    prepare_data=prepare_data_noref
)

In [80]:
qa_evaluator = LangChainStringEvaluator(
    "qa",
    config={"llm": eval_llm},
    prepare_data=lambda run, example: {
        "prediction": run.outputs["response"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)  

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["response"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm": eval_llm 
    },
    prepare_data=lambda run, example: {
       "prediction": run.outputs["response"],
       "input": example.inputs["question"],
    }
)

## LangSmith Evaluation

## Dope-ifying Our Application

In [81]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [83]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(filtered_docs)

In [84]:
from langchain_openai import OpenAIEmbeddings

large_embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [85]:
try:
    vectorstore.client.create_collection(
      collection_name="Loan Data for RAG",
      vectors_config=models.VectorParams(size=3072, distance=models.Distance.COSINE) ### was 1536
    )
except:
    pass

dope_app_vectorstore = Qdrant(
  client=vectorstore.client,     # ✅ Reuse existing client
  embeddings=large_embeddings,         # ✅ Reuse embeddings
  collection_name="Loan Data for RAG"
)

# Add documents after creation
_ = dope_app_vectorstore.add_documents(rag_documents)

In [86]:
retriever = vectorstore.as_retriever()

In [87]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

In [88]:
gc.collect()

1802

## Evaluation and Performance Analysis

Now that we have evaluation data from LangSmith, let's analyze the performance of different retrievers across multiple dimensions: **Performance**, **Cost**, and **Latency**.

In [89]:
from tqdm.notebook import tqdm

In [90]:
pipeline_stages_folder_name = ".pipeline-stages"
os.makedirs(pipeline_stages_folder_name, exist_ok=True)
def write_to_file(filename: str, content: str):
    with open(f"{pipeline_stages_folder_name}/{filename}", 'w') as text_file:
        try:
            text_file.write(content)
        finally:
            text_file.close()

In [91]:
from evaluation_cache import save_evaluation_result, load_evaluation_result

In [117]:
%%time
retriever_chains_list = {
    "naive_retrieval_chain" : naive_retrieval_chain,
    "bm25_retrieval_chain": bm25_retrieval_chain,
    "contextual_compression_retrieval_chain": contextual_compression_retrieval_chain,
    "multi_query_retrieval_chain": multi_query_retrieval_chain,
    "parent_document_retrieval_chain": parent_document_retrieval_chain,
    "ensemble_retrieval_chain": ensemble_retrieval_chain,
}

evaluation_results = {}
retriever_eval_progress_bar = tqdm(retriever_chains_list)
for retriever_chain in retriever_eval_progress_bar:
    gc.collect()
        
    evaluation_results_filename = f"{pipeline_stages_folder_name}/evaluation_results_{retriever_chain}.pkl"
    if os.path.exists(evaluation_results_filename):
        print(f"{retriever_chain} already processed, skipping to the next one...")
        evaluation_results[retriever_chain] = load_evaluation_result(evaluation_results_filename)
        continue

    retriever_eval_progress_bar.set_description(retriever_chain, refresh=True)
    chain_to_invoke = retriever_chains_list[retriever_chain]
    try:
        result = evaluate(
          chain_to_invoke.invoke,
          data=dataset_name,
          evaluators=[
              cot_qa_evaluator, 
              qa_evaluator, 
              labeled_helpfulness_evaluator, 
              empathy_evaluator, 
              unlabeled_dopeness_evaluator
          ],
          num_repetitions=3,
          max_concurrency=1,  # Limit concurrent requests to avoid rate limits
          metadata={"revision_id": retriever_chain},
          experiment_prefix=retriever_chain
        )

        # Store the result for analysis
        print(f"Saving {retriever_chain}...")
        save_evaluation_result(result, evaluation_results_filename)
        evaluation_results[retriever_chain] = result
        
        print(f"Finished evaluating and saving {retriever_chain} moving to the next one...")
    except Exception as ex:
        print(f"Failed to run evaluation on the {retriever_chain}, due to {ex}, skipping to the next one...")
        continue    

  0%|          | 0/6 [00:00<?, ?it/s]

naive_retrieval_chain already processed, skipping to the next one...
✅ Loaded evaluation results from: .pipeline-stages/evaluation_results_naive_retrieval_chain.pkl
📅 Cached on: 2025-07-28T22:58:12.623682
bm25_retrieval_chain already processed, skipping to the next one...
✅ Loaded evaluation results from: .pipeline-stages/evaluation_results_bm25_retrieval_chain.pkl
📅 Cached on: 2025-07-28T21:53:03.466554
contextual_compression_retrieval_chain already processed, skipping to the next one...
✅ Loaded evaluation results from: .pipeline-stages/evaluation_results_contextual_compression_retrieval_chain.pkl
📅 Cached on: 2025-07-28T21:56:33.317981
multi_query_retrieval_chain already processed, skipping to the next one...
✅ Loaded evaluation results from: .pipeline-stages/evaluation_results_multi_query_retrieval_chain.pkl
📅 Cached on: 2025-07-28T22:09:40.534506
parent_document_retrieval_chain already processed, skipping to the next one...
✅ Loaded evaluation results from: .pipeline-stages/evalua

### Analysis Framework

The performance analysis script provides comprehensive evaluation across three key dimensions:

#### 📊 **Performance Metrics**
- **QA Score**: Correctness of answers based on reference ground truth
- **Helpfulness Score**: How helpful responses are to users
- **Empathy Score**: Empathetic quality of responses

#### 💰 **Cost Analysis**
- **Total Cost**: Cumulative cost across all evaluation runs
- **Cost per Run**: Average cost per individual evaluation
- **Token Usage**: Input and output token consumption

#### ⚡ **Latency Analysis**
- **Average Latency**: Mean response time per retriever
- **Total Processing Time**: Cumulative time across evaluations
- **Latency Distribution**: Variation in response times

#### 🎯 **Overall Ranking**
Uses weighted scoring:
- **40%** Performance (accuracy/quality)
- **30%** Cost efficiency 
- **30%** Speed/latency

This provides a balanced view for different use cases and requirements.

## 🎯 Direct Analysis from Evaluate Results

The most accurate approach is to analyze the results returned directly from the `evaluate()` function calls. This gives us immediate access to all metrics without needing to query LangSmith again.

### Analyze Collected Results

In [ ]:
from performance_analysis import analyze_retrievers
df = analyze_retrievers(evaluation_results)

In [121]:
summary_df = df[[
    'Retriever',
    'Correctness_Avg_Score', 
    'QA_Avg_Score',
    'Helpfulness_Avg_Score',
    'Empathy_Avg_Score',
    'Avg_Latency_Sec',
    'Total_Runs',
    'Avg_Input_Tokens_Per_Run',
    'Avg_Output_Tokens_Per_Run',
    'Total_Cost', 
    'Total_Latency_Sec'
]].round(4)
summary_df

,Retriever,Correctness_Avg_Score,QA_Avg_Score,Helpfulness_Avg_Score,Empathy_Avg_Score,Avg_Latency_Sec,Total_Runs,Avg_Input_Tokens_Per_Run,Avg_Output_Tokens_Per_Run,Total_Cost,Total_Latency_Sec
1,Bm25,0.900,0.0,0.667,0.000,0.0,30,3846.0000,270.7000,0.0039,0.0
2,Contextual Compression,0.900,0.0,0.800,0.000,0.0,30,792.2333,83.5000,0.0009,0.0
5,Ensemble,0.867,0.0,0.900,0.000,0.0,30,17408.7667,356.8667,0.0163,0.0
0,Naive,0.833,0.0,0.900,0.033,0.0,30,7877.3000,308.2667,0.0076,0.0
4,Parent Document,0.833,0.0,0.833,0.000,0.0,30,4256.2000,290.3667,0.0044,0.0
3,Multi Query,0.828,0.0,0.867,0.033,0.0,30,11298.5667,326.1667,0.0108,0.0


In [122]:
df.to_csv('langsmith_retriever_raw_stats.csv', index=False)
summary_df.to_csv('langsmith_retriever_summary_stats.csv', index=False)

In [160]:
import importlib
import langsmith_rank_retrievers 
importlib.reload(langsmith_rank_retrievers)

from langsmith_rank_retrievers import get_balanced_summary, get_balanced_detailed, get_all_scenarios_comparison

## Final outcome of the Langsmith Evaluators

![Langsmith-retrievers-stats-1-of-2.jpg](Langsmith-retrievers-stats-1-of-2.jpg) | ![Langsmith-retrievers-stats-2-of-2.jpg](Langsmith-retrievers-stats-2-of-2.jpg)

In [161]:
get_all_scenarios_comparison()

,Scenario,1st Place,2nd Place,3rd Place
0,BALANCED,Contextual Compression,Parent Document,Bm25
1,QUALITY_FIRST,Contextual Compression,Naive,Parent Document
2,COST_CONSCIOUS,Contextual Compression,Bm25,Parent Document


In [162]:
get_balanced_summary()

,Rank,Retriever,Final_Score,Quality_Score,Efficiency_Score
1,1,Contextual Compression,0.554946,0.42500,0.944785
4,2,Parent Document,0.494890,0.41650,0.730061
0,3,Bm25,0.483997,0.39175,0.760736
3,4,Naive,0.464561,0.44150,0.533742
5,5,Multi Query,0.408356,0.43200,0.337423
2,6,Ensemble,0.331312,0.44175,0.000000


In [163]:
get_balanced_detailed()

,Rank,Retriever,Correctness_Avg_Score,Helpfulness_Avg_Score,Empathy_Avg_Score,Total_Cost
1,1,Contextual Compression,0.900,0.800,0.000,0.0009
4,2,Parent Document,0.833,0.833,0.000,0.0044
0,3,Bm25,0.900,0.667,0.000,0.0039
3,4,Naive,0.833,0.900,0.033,0.0076
5,5,Multi Query,0.828,0.867,0.033,0.0108
2,6,Ensemble,0.867,0.900,0.000,0.0163
